In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import getcwd
from UliPlot.XLSX import auto_adjust_xlsx_column_width
from datetime import datetime

In [115]:
pmsystem_project_list_file = '(Input) ProjectListExport.xls'
cft_list_file = '(Input) CFTListExport.xls'
prweb_file = '(Input) New Production Status Summary.xlsx'

start_from = '2022-08'

consumer_dpqe_list = ['Lo, Aalto','Kuo, Chris','Wang, Cooper', 'Fu, Ernest','Hsu, Jason(PQE)','Ni, Wendy','Wang, Evelyn','Hsu, Jason(PQE);\nWang, Evelyn'] 
# 'Hsu, Jason(PQE);\nWang, Evelyn' is for GPSMAP 9000 series

# PM system

Read

In [116]:
pmsystem_project = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,3,4,5,6,9,10,12], parse_dates=[2,3,4,5], index_col=[0])
pmsystem_cft = pd.read_excel(cft_list_file, usecols=[0,44], index_col=[0])

c:\Users\wangcooper\Miniconda3\envs\workspace\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [117]:
pmsystem_cft.head()

,Develop Product QE (DPQE)
Project ID,
15624,"Lo, Aalto"
16002,"Lo, Aalto"
16263,"Lo, Aalto"
14004,"Lo, Aalto"
15505,"Lo, Aalto"


In [118]:
pmsystem_project = pmsystem_project[(pmsystem_project['PDP/Form107'] == 'PDP')]

In [119]:
pmsystem_project.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status
Project ID,,,,,,,,
15624,Catalyst Kart/Moto/Auto,2022-06-24,2023-01-09,2023-07-24,2023-05-10,Global,PDP,Development
13009,Dash Cam Live (LTE),2021-06-25,2022-02-25,2022-11-21,2022-08-24,Global,PDP,Development
14984,"Dash Cam Live (LTE), ASIA",NaT,2022-03-31,2023-02-13,2022-12-09,Asia,PDP,Development
16199,DC-2023 Premium Dash Cameras,2023-02-02,2023-05-10,2023-08-14,2023-06-15,Global,PDP,Development
16547,"DC-2023 Premium Dash Cameras, Asia",NaT,NaT,2023-09-04,2023-05-31,Asia,PDP,Development


Merge project and CFT lists

In [120]:
pmsystem_project_cft = pd.merge(left=pmsystem_project, right=pmsystem_cft, left_index=True, right_index=True)
pmsystem_project_cft.sort_index().head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
5711,Product - GLA 85,2014-10-30,NaT,2018-12-07,2016-08-31,Global,PDP,Development,"Kuo, Chris"
7011,Xero G2i SatCom Scouting Camera,NaT,NaT,NaT,NaT,Global,PDP,Development,"Kuo, Chris"
7029,Alpha Dog Collar,2021-11-18,2022-08-18,2023-03-01,2022-12-16,Global,PDP,Development,"Kuo, Chris"
7563,Mercury,NaT,NaT,2022-01-17,2021-04-23,Global,PDP,Development,"Fu, Ernest"


Find projects which WW/ASIA SKU have different DPQE

In [121]:
ww_asia_diff = pmsystem_project_cft.copy()

In [122]:
#Handle Asis words in 'Project Name' column
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].apply(lambda name : name.split(',')[0])
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('ASIA','')
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('Asia','')

In [123]:
# check any missing
ww_asia_diff[ww_asia_diff['Project Name'].str.contains('ASIA|asia|Asia')]

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,


In [124]:
ww_asia_diff = ww_asia_diff[['Project Name','Develop  Product QE (DPQE)','Global/Asia','MP Date']].loc[ww_asia_diff['MP Date']> start_from]

In [125]:
# Find duplicated projects
ww_asia_diff = ww_asia_diff[ww_asia_diff.duplicated(subset=['Project Name'],keep=False)]
ww_asia_diff

,Project Name,Develop Product QE (DPQE),Global/Asia,MP Date
Project ID,,,,
10901,eTrex Solar,"Kuo, Chris",Global,2023-08-21
10926,Approach R50,"Kuo, Chris",Global,2023-12-25
12799,Descent S1 Buoy,"Lo, Aalto",Global,2023-07-31
12994,Descent Mk3,"Wang, Cooper",Global,2023-10-02
13009,Dash Cam Live (LTE),"Kuo, Chris",Global,2022-11-21
13084,Instinct Crossover,"Lo, Aalto",Global,2022-10-11
14004,vivomove Trend,"Lo, Aalto",Global,2022-11-21
14069,Descent S1 Buoy,"Lo, Aalto",Asia,2023-07-31
14984,Dash Cam Live (LTE),"Kuo, Chris",Asia,2023-02-13


In [126]:
# Find projects have different DPQE
ww_asia_diff_DPQE = ww_asia_diff[~ww_asia_diff.duplicated(subset=['Project Name','Develop  Product QE (DPQE)'],keep=False)]

Find projects w/o DPQE (include ASIA SKUs)

In [127]:
projects_wo_DPQE = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isnull()].dropna(thresh=5, axis=0).loc[(pmsystem_project_cft['MP Date']>start_from)]

Find projects with multi-DPQE (include ASIA SKUs)

In [128]:
multi_dpqe = pmsystem_project_cft[['Project Name','Global/Asia','Develop  Product QE (DPQE)','MP Date']]
multi_dpqe = multi_dpqe[multi_dpqe['MP Date']>start_from]
multi_dpqe.dropna(inplace=True)

In [129]:
multi_dpqe = multi_dpqe[multi_dpqe['Develop  Product QE (DPQE)'].str.contains('\n')]

Find projects w/o any date

In [130]:
# Find all date is NaT
# 1. Select dataframe w/ date (pmsystem_project_cft.iloc[:,1:5])
# 2. Rows w/ any date marked as True (notnull().any(axis=1))
# 3. Reverse (~)
pmsystem_project_wo_any_date = pmsystem_project_cft.loc[(~pmsystem_project_cft.iloc[:,1:5].notnull().any(axis=1))].sort_values(['Develop  Product QE (DPQE)','Project Name']).set_index('Develop  Product QE (DPQE)')
pmsystem_project_wo_any_date

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status
Develop Product QE (DPQE),,,,,,,,
"Fu, Ernest",SaMD PPG Atrial Fibrillation Detection,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Fu, Ernest",SaMD Sleep Apnea Risk Assessment,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Hsu, Jason(PQE)",Panoptix PS22-IF Transducer and Ice Fishing Bu...,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",Approach G20,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",Companion Dog Tracking Collar,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",Connect IQ 3.x,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",Edge 540 840 ASIA,NaT,NaT,NaT,NaT,Asia,PDP,Development
"Kuo, Chris",L20 Laser Range Finder,NaT,NaT,NaT,NaT,Global,PDP,Concepts
"Kuo, Chris",Rally 110/210 Family,NaT,NaT,NaT,NaT,Global,PDP,Concepts


Melt

In [131]:
df_consumer_dpqe = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isin(consumer_dpqe_list)].sort_index()
df_consumer_dpqe = df_consumer_dpqe[df_consumer_dpqe['Global/Asia']=='Global']
df_consumer_dpqe.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
5711,Product - GLA 85,2014-10-30,NaT,2018-12-07,2016-08-31,Global,PDP,Development,"Kuo, Chris"
7011,Xero G2i SatCom Scouting Camera,NaT,NaT,NaT,NaT,Global,PDP,Development,"Kuo, Chris"
7029,Alpha Dog Collar,2021-11-18,2022-08-18,2023-03-01,2022-12-16,Global,PDP,Development,"Kuo, Chris"
7563,Mercury,NaT,NaT,2022-01-17,2021-04-23,Global,PDP,Development,"Fu, Ernest"


In [132]:
value_vars = ['Working Prototype Built and Tested','Final Factory Prototypes Built','PR Date','MP Date']
pmsystem_project_dpqe_melted = df_consumer_dpqe.reset_index().melt(id_vars= ['Project ID','Project Name','Develop  Product QE (DPQE)'],value_vars=value_vars)
pmsystem_project_dpqe_melted.head()

,Project ID,Project Name,Develop Product QE (DPQE),variable,value
0,2854,Product - GDC 7500,"Kuo, Chris",Working Prototype Built and Tested,2013-04-30
1,5711,Product - GLA 85,"Kuo, Chris",Working Prototype Built and Tested,2014-10-30
2,7011,Xero G2i SatCom Scouting Camera,"Kuo, Chris",Working Prototype Built and Tested,NaT
3,7029,Alpha Dog Collar,"Kuo, Chris",Working Prototype Built and Tested,2021-11-18
4,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaT


In [133]:
pmsystem_project_dpqe_melted.rename(columns={'Develop  Product QE (DPQE)':'DPQE','variable':'Phase','value':'Date'},inplace=True)

In [134]:
pmsystem_project_dpqe_melted.Date = pmsystem_project_dpqe_melted.Date.dt.strftime('%Y-%m')

In [135]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted[pmsystem_project_dpqe_melted.Date >= start_from]

In [136]:
pmsystem_project_dpqe_melted.isnull().sum()

Project ID      0
Project Name    0
DPQE            0
Phase           0
Date            0
dtype: int64

Weight socre

In [137]:
pmsystem_project_dpqe_melted.loc[:,'weighted_score'] = pmsystem_project_dpqe_melted.loc[:,'Phase'].apply(lambda phase : 1 if (phase == 'Working Prototype Built and Tested') | (phase == 'MP Date') else 2 if phase=='Final Factory Prototypes Built' else 3 if phase=='PR Date' else 0)

In [138]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted.rename(columns={"Project ID":'PM Project Id','Project Name':'PM Project Name'})

In [139]:
pmsystem_project_dpqe_melted.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score
6,8510,Xero L60i Laser Navigator,"Kuo, Chris",Working Prototype Built and Tested,2022-10,1
12,10617,Force Kayak Propulsion System,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2023-07,1
15,10901,eTrex Solar,"Kuo, Chris",Working Prototype Built and Tested,2022-12,1
17,10926,Approach R50,"Kuo, Chris",Working Prototype Built and Tested,2023-01,1
19,11778,Tire Temp Sensor,"Ni, Wendy",Working Prototype Built and Tested,2022-08,1


# PRweb

Read

In [140]:
prweb = pd.read_excel(prweb_file, header=1, usecols=[0,3])
prweb = prweb[prweb['Project Name'] != 'GPSMAP 9000_Black Box'] # 2022/09/05, GPSMAP 9500 replace GPSMAP 9000_Black Box
prweb.head()

,Project Name,PM Project Id
0,Rally X10 Family,16349
1,Force Kayak Propulsion System,10617
2,Xero B60i Laser Navigation Binoculars,13229
3,TGWW25,16227
4,LYWW24,15976


Merge with data from PMsystem

In [141]:
prweb_pmsystem = pd.merge(left=pmsystem_project_dpqe_melted, right=prweb, how='left', left_on='PM Project Id', right_on='PM Project Id', suffixes=['_PMsystem','_prweb'])

In [142]:
prweb_pmsystem.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
0,8510,Xero L60i Laser Navigator,"Kuo, Chris",Working Prototype Built and Tested,2022-10,1,Xero L60i Laser Navigator
1,10617,Force Kayak Propulsion System,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2023-07,1,Force Kayak Propulsion System
2,10901,eTrex Solar,"Kuo, Chris",Working Prototype Built and Tested,2022-12,1,eTrex Solar
3,10926,Approach R50,"Kuo, Chris",Working Prototype Built and Tested,2023-01,1,Approach R50
4,11778,Tire Temp Sensor,"Ni, Wendy",Working Prototype Built and Tested,2022-08,1,Tire Temp Sensor


Handle GPSMAP 9000 19"

In [143]:
'Hsu, Jason(PQE);\nWang, Evelyn'.replace('\n','').split(';')

['Hsu, Jason(PQE)', 'Wang, Evelyn']

In [144]:
# GPSMAP 9000 - 19" to Wang, Evelyn, others to Hsu, Jason(PQE)
def gpsmap_9000 (project_name, DPQE):
    if project_name == 'GPSMAP 9000 - 19"':
        return DPQE.replace('\n','').split(';')[1] #Wang, Evelyn
    elif project_name in ['GPSMAP 9000 - 22"','GPSMAP 9000 - 24"','GPSMAP 9000 -27"','GPSMAP 9000_Black Box']:
        return DPQE.replace('\n','').split(';')[0] #Hsu, Jason(PQE)
    else:
        return DPQE


In [145]:
prweb_pmsystem['DPQE'] = np.vectorize(gpsmap_9000)(prweb_pmsystem['Project Name'],prweb_pmsystem['DPQE'])

In [146]:
#Validate
prweb_pmsystem[prweb_pmsystem['PM Project Name'] == 'GPSMAP 9000']

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
35,9946,GPSMAP 9000,"Wang, Evelyn",Final Factory Prototypes Built,2023-02,2,"GPSMAP 9000 - 19"""
36,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Final Factory Prototypes Built,2023-02,2,"GPSMAP 9000 - 24"""
37,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Final Factory Prototypes Built,2023-02,2,"GPSMAP 9000 -27"""
38,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Final Factory Prototypes Built,2023-02,2,"GPSMAP 9000 - 22"""
88,9946,GPSMAP 9000,"Wang, Evelyn",PR Date,2023-05,3,"GPSMAP 9000 - 19"""
89,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-05,3,"GPSMAP 9000 - 24"""
90,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-05,3,"GPSMAP 9000 -27"""
91,9946,GPSMAP 9000,"Hsu, Jason(PQE)",PR Date,2023-05,3,"GPSMAP 9000 - 22"""
152,9946,GPSMAP 9000,"Wang, Evelyn",MP Date,2023-07,1,"GPSMAP 9000 - 19"""
153,9946,GPSMAP 9000,"Hsu, Jason(PQE)",MP Date,2023-07,1,"GPSMAP 9000 - 24"""


# Detail

In [147]:
detail = prweb_pmsystem.groupby(['DPQE','Project Name','Date']).agg({'weighted_score':'sum','Project Name':'count'}).sort_index(level='Date').rename(columns=({'weighted_score':'Weighted_Project_counts','Project Name':'Project_counts'})).unstack(fill_value=0).style.background_gradient(cmap="OrRd", axis=None)

# Style overview DataFrame

Overview

In [148]:
overview = prweb_pmsystem.groupby(['DPQE','Date']).count()['Project Name'].sort_index(level='Date').unstack(fill_value=0).style.background_gradient(cmap="OrRd", axis=None)
overview

Date,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-07,2024-08,2024-09,2024-11
DPQE,,,,,,,,,,,,,,,,,,,,,,,,,
"Fu, Ernest",1,4,3,3,2,2,3,1,2,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0
"Hsu, Jason(PQE)",0,1,5,6,4,3,6,2,5,4,3,5,3,1,2,1,0,0,0,0,1,0,1,0,1
"Kuo, Chris",2,0,4,2,4,2,0,4,1,1,1,3,2,2,0,1,1,1,0,1,0,1,0,2,1
"Lo, Aalto",2,0,3,4,1,6,1,1,2,3,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0
"Ni, Wendy",2,2,1,1,1,1,3,1,2,1,3,0,1,0,0,0,1,0,1,0,0,0,0,0,0
"Wang, Cooper",3,2,4,1,4,2,2,5,3,2,0,2,1,1,2,0,1,0,0,0,0,0,0,0,0
"Wang, Evelyn",0,0,1,1,0,1,1,3,0,2,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0


Overview_weighted

In [149]:
overview_weighted = prweb_pmsystem.groupby(['DPQE','Date']).sum()['weighted_score'].sort_index(level='Date').unstack(fill_value=0).style.background_gradient(cmap="OrRd", axis=None)
overview_weighted

Date,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-07,2024-08,2024-09,2024-11
DPQE,,,,,,,,,,,,,,,,,,,,,,,,,
"Fu, Ernest",1,6,5,8,5,2,8,1,3,1,3,4,0,1,1,0,0,0,0,0,0,0,0,0,0
"Hsu, Jason(PQE)",0,2,9,14,4,7,18,3,6,12,10,6,7,3,2,1,0,0,0,0,2,0,3,0,1
"Kuo, Chris",5,0,7,2,8,3,0,7,2,1,3,9,2,2,0,3,1,1,0,2,0,3,0,4,1
"Lo, Aalto",4,0,5,6,2,12,3,1,4,9,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0
"Ni, Wendy",3,4,2,3,3,1,7,1,2,2,5,0,1,0,0,0,3,0,1,0,0,0,0,0,0
"Wang, Cooper",7,2,7,1,7,6,4,9,4,3,0,5,3,1,4,0,1,0,0,0,0,0,0,0,0
"Wang, Evelyn",0,0,2,1,0,3,2,4,0,6,1,1,2,0,0,3,0,1,0,0,0,0,0,0,0


# 2022/09/02 request by Ray

 Find PM system projects MP dates from NOW

In [150]:
pmsystem_project2 = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,5,8,9,10,12], 
    parse_dates=[2,3,4,5], engine='openpyxl')
pmsystem_project2 = pmsystem_project2[pmsystem_project2['Global/Asia']=='Global']
pmsystem_project2.head()

c:\Users\wangcooper\Miniconda3\envs\workspace\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Project ID,Project Name,MP Date,Category,Global/Asia,PDP/Form107,Status
0,15624,Catalyst Kart/Moto/Auto,2023-07-24,Automotive,Global,PDP,Development
1,13009,Dash Cam Live (LTE),2022-11-21,Automotive,Global,PDP,Development
3,16199,DC-2023 Premium Dash Cameras,2023-08-14,Automotive,Global,PDP,Development
5,16249,dezl headset (2nd Gen),2024-02-12,Automotive,Global,PDP,Development
6,16467,Drive 53,NaT,Automotive,Global,PDP,Development


In [151]:
MPdate_from_now = pmsystem_project2[pmsystem_project2['MP Date'] > datetime.now()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
MPdate_from_now['MP Date'] = MPdate_from_now['MP Date'].dt.date
MPdate_from_now

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,13009,Dash Cam Live (LTE),2022-11-21,Global,PDP,Development
Automotive,15955,"Tread 8"" OEM - Honda",2023-01-09,Global,PDP,Development
Automotive,16069,zumo XT2,2023-03-06,Global,PDP,Development
Automotive,11778,Tire Temp Sensor,2023-04-10,Global,PDP,Development
Automotive,15624,Catalyst Kart/Moto/Auto,2023-07-24,Global,PDP,Development
...,...,...,...,...,...,...
Outdoor,8510,Xero L60i Laser Navigator,2023-09-18,Global,PDP,Development
Outdoor,12994,Descent Mk3,2023-10-02,Global,PDP,Development
Outdoor,10926,Approach R50,2023-12-25,Global,PDP,Development


In [152]:
MPdate_from_now_count = MPdate_from_now.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
MPdate_from_now_count

,Count
Category,
Automotive,7
Fitness,12
Marine,19
OEM Auto,1
Outdoor,23


In [153]:
No_MPdate = pmsystem_project2[pmsystem_project2['MP Date'].isnull()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
No_MPdate = No_MPdate[~No_MPdate.index.isin(['Aviation','Garmin Labs'])]
No_MPdate

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,16467,Drive 53,NaT,Global,PDP,Development
Automotive,16544,2024 Value Dash Cams,NaT,Global,PDP,Concepts
Automotive,16319,Aviation Headset -,NaT,Global,PDP,Concepts
Automotive,16718,DEZL 2024,NaT,Global,PDP,Concepts
Automotive,16717,Drive Series 2024,NaT,Global,PDP,Concepts
...,...,...,...,...,...,...
Outdoor,16037,M60i Laser Navigation Binoculars 15/18x56mm,NaT,Global,PDP,Concepts
Outdoor,16212,Marq (Gen 3),NaT,Global,PDP,Concepts
Outdoor,12279,Marq Hybrid,NaT,Global,PDP,Concepts


In [154]:
No_MPdate_count = No_MPdate.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
No_MPdate_count

,Count
Category,
Automotive,7
Fitness,11
Marine,11
OEM Auto,21
Outdoor,28


Find Avation projects MP dates from PRweb

In [155]:
prweb2 = pd.read_excel(prweb_file, header=1, usecols=[0,1,5,6])
Aviation_PRweb = prweb2[prweb2['Category'] == 'Aviation'].sort_values('MP Date')
Aviation_PRweb

,Project Name,Project Id,Category,MP Date
37,Product - G3X/G500TXi Hybrid,1050,Aviation,2023-07-01
38,"Product - 1.3\"" GTR/GNC",984,Aviation,2023-07-01
9,Product - GDU147XW,1049,Aviation,2024-01-03
119,Product - Flight Stream Redesign,977,Aviation,NaN
120,Product - GDU77XW,1048,Aviation,NaN


# to_excel

In [156]:
with pd.ExcelWriter(getcwd()+'\(Output) Data.xlsx', engine='openpyxl') as writer:

    overview.to_excel(writer, sheet_name='overview')

    overview_weighted.to_excel(writer, sheet_name='overview_weighted')
    
    detail.to_excel(writer, sheet_name='detail')
    #auto_adjust_xlsx_column_width(overall, writer, sheet_name="overall", margin=5)
    
    pmsystem_project_wo_any_date.to_excel(writer, sheet_name='pmsystem_project_wo_any_date')
    auto_adjust_xlsx_column_width(pmsystem_project_wo_any_date, writer, sheet_name="pmsystem_project_wo_any_date", margin=5)
    
    ww_asia_diff_DPQE.to_excel(writer, sheet_name='ww_asia_diff_DPQE')
    auto_adjust_xlsx_column_width(ww_asia_diff_DPQE, writer, sheet_name="ww_asia_diff_DPQE", margin=5)
    
    projects_wo_DPQE.to_excel(writer, sheet_name='projects_wo_DPQE')
    auto_adjust_xlsx_column_width(projects_wo_DPQE, writer, sheet_name="projects_wo_DPQE", margin=5)
    
    multi_dpqe.to_excel(writer, sheet_name='multi_dpqe')
    auto_adjust_xlsx_column_width(multi_dpqe, writer, sheet_name="multi_dpqe", margin=5)

    MPdate_from_now_count.to_excel(writer, sheet_name='MPdate_from_now_count')
    auto_adjust_xlsx_column_width(MPdate_from_now_count, writer, sheet_name="MPdate_from_now_count", margin=3)
    
    MPdate_from_now.to_excel(writer, sheet_name='MPdate_from_now')
    auto_adjust_xlsx_column_width(MPdate_from_now, writer, sheet_name="MPdate_from_now", margin=3)
    
    No_MPdate_count.to_excel(writer, sheet_name='No_MPdate_count')
    auto_adjust_xlsx_column_width(No_MPdate_count, writer, sheet_name="No_MPdate_count", margin=3)
    
    No_MPdate.to_excel(writer, sheet_name='No_MPdate')
    auto_adjust_xlsx_column_width(No_MPdate, writer, sheet_name="No_MPdate", margin=3)

    Aviation_PRweb.to_excel(writer, sheet_name='Aviation_PRweb')
    auto_adjust_xlsx_column_width(Aviation_PRweb, writer, sheet_name="Aviation_PRweb", margin=3)

